In [ ]:
#! pip3 install --upgrade --user google-cloud-aiplatform

In [15]:
import re
import pandas as pd
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai.preview.generative_models as generative_models 
import sys
import os
from dotenv import load_dotenv #en el env esta la api

In [4]:
# Define project information
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
model = GenerativeModel("gemini-1.5-flash-001")

In [6]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [7]:
generation_config =GenerationConfig(
    temperature=0,
    top_p=0.8,
    max_output_tokens=8192,
)

In [8]:
import csv

# Leer el archivo CSV en una lista de listas
with open('club_salaries.csv', mode='r') as file:
    reader = csv.reader(file)
    data = [row for row in reader]

# Ahora data es una variable que contiene la lista de listas
print(data)

[['club_name', 'url', 'element_id', 'value'], ['', 'https://www.capology.comClub', 'weekly-avg', 'Error: Message: unknown error: net::ERR_NAME_NOT_RESOLVED\n  (Session info: chrome=128.0.6613.114)\nStacktrace:\n\tGetHandleVerifier [0x005C8213+26163]\n\t(No symbol) [0x00559CC4]\n\t(No symbol) [0x004524C3]\n\t(No symbol) [0x0044AAD1]\n\t(No symbol) [0x0043E668]\n\t(No symbol) [0x0044008B]\n\t(No symbol) [0x0043E8B7]\n\t(No symbol) [0x0043E25C]\n\t(No symbol) [0x0043E188]\n\t(No symbol) [0x0043C588]\n\t(No symbol) [0x0043CB1B]\n\t(No symbol) [0x00454F04]\n\t(No symbol) [0x004D5107]\n\t(No symbol) [0x004BACBC]\n\t(No symbol) [0x004D4661]\n\t(No symbol) [0x004BAA56]\n\t(No symbol) [0x0048BE89]\n\t(No symbol) [0x0048C8CD]\n\tGetHandleVerifier [0x0089D313+2996019]\n\tGetHandleVerifier [0x008F1B89+3342249]\n\tGetHandleVerifier [0x00657AEF+614159]\n\tGetHandleVerifier [0x0065F17C+644508]\n\t(No symbol) [0x005627FD]\n\t(No symbol) [0x0055F6F8]\n\t(No symbol) [0x0055F895]\n\t(No symbol) [0x00551C

In [9]:
# Leer el archivo CSV en una variable como texto
with open('club_salaries.csv', mode='r') as file:
    text = file.read()

# Ahora text es una variable que contiene el contenido del archivo CSV como un string
print(text)


club_name,url,element_id,value
,https://www.capology.comClub,weekly-avg,"Error: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=128.0.6613.114)
Stacktrace:
	GetHandleVerifier [0x005C8213+26163]
	(No symbol) [0x00559CC4]
	(No symbol) [0x004524C3]
	(No symbol) [0x0044AAD1]
	(No symbol) [0x0043E668]
	(No symbol) [0x0044008B]
	(No symbol) [0x0043E8B7]
	(No symbol) [0x0043E25C]
	(No symbol) [0x0043E188]
	(No symbol) [0x0043C588]
	(No symbol) [0x0043CB1B]
	(No symbol) [0x00454F04]
	(No symbol) [0x004D5107]
	(No symbol) [0x004BACBC]
	(No symbol) [0x004D4661]
	(No symbol) [0x004BAA56]
	(No symbol) [0x0048BE89]
	(No symbol) [0x0048C8CD]
	GetHandleVerifier [0x0089D313+2996019]
	GetHandleVerifier [0x008F1B89+3342249]
	GetHandleVerifier [0x00657AEF+614159]
	GetHandleVerifier [0x0065F17C+644508]
	(No symbol) [0x005627FD]
	(No symbol) [0x0055F6F8]
	(No symbol) [0x0055F895]
	(No symbol) [0x00551C16]
	BaseThreadInitThunk [0x76E77BA9+25]
	RtlInitializeExceptionChain [0x7783C1

In [17]:

# Prompt con el contexto y la pregunta
prompt = f"""Quisiera que extraigas del siguiente texto: {text} la informacion relevante que te voy a explicar a continuacion:
Debes colocar en una tabla en la primera columna el nombre del club que esta en el link del texto, en la segunda columna el valor numerico
de annual-avg y en la tercer columna el valor numerico de weekly-avg, debes realizar esto con todos los clubes en el texto.
Utiliza informacion unicamente contenida en este texto"""

# Generar la respuesta utilizando el prompt actualizado
responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
full_text = ""
for response in responses:
    full_text += response.text

print(full_text)

## Salarios de clubes de fútbol en Argentina (2024)

| Club | Annual Avg | Weekly Avg |
|---|---|---|
| Argentinos Juniors | $181,724 | $3,495 |
| Banfield | $171,935 | $3,306 |
| Barracas | $181,000 | $3,481 |
| Belgrano | $239,000 | $4,596 |
| Boca Juniors | $708,824 | $13,631 |
| Central Córdoba | $161,143 | $3,099 |
| Defensa y Justicia | $196,774 | $3,784 |
| Estudiantes | $543,462 | $10,451 |
| Gimnasia La Plata | $230,000 | $4,423 |
| Godoy Cruz | $208,000 | $4,000 |
| Huracán | $313,333 | $6,026 |
| Independiente | $285,882 | $5,498 |
| Instituto | $246,000 | $4,731 |
| Lanús | $374,800 | $7,208 |
| Newell's Old Boys | $289,615 | $5,570 |
| Platense | $174,063 | $3,347 |
| Racing Club | $525,714 | $10,110 |
| Riestra | $111,515 | $2,145 |
| Rivadavia | $148,125 | $2,849 |
| River Plate | $987,656 | $18,993 |
| Rosario Central | $319,655 | $6,147 |
| San Lorenzo | $409,615 | $7,877 |
| Sarmiento | $237,742 | $4,572 |
| Talleres | $320,714 | $6,168 |
| Tigre | $203,103 | $3,906 |

In [18]:
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_text)


# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df = pd.DataFrame(matches)#, columns=['Jugador', 'Percentil Prom Def', 'Percentil Prom Org', 'Rol Score'])

df = df[~df.eq('---').all(axis=1)]
df.columns = df.iloc[0]  # La primera fila se convierte en nombres de columna
df = df[1:]
# Resetea el índice
df.reset_index(drop=True, inplace=True)

print(df)

0                 Club Annual Avg Weekly Avg
0   Argentinos Juniors   $181,724     $3,495
1             Banfield   $171,935     $3,306
2             Barracas   $181,000     $3,481
3             Belgrano   $239,000     $4,596
4         Boca Juniors   $708,824    $13,631
5      Central Córdoba   $161,143     $3,099
6   Defensa y Justicia   $196,774     $3,784
7          Estudiantes   $543,462    $10,451
8    Gimnasia La Plata   $230,000     $4,423
9           Godoy Cruz   $208,000     $4,000
10             Huracán   $313,333     $6,026
11       Independiente   $285,882     $5,498
12           Instituto   $246,000     $4,731
13               Lanús   $374,800     $7,208
14   Newell's Old Boys   $289,615     $5,570
15            Platense   $174,063     $3,347
16         Racing Club   $525,714    $10,110
17             Riestra   $111,515     $2,145
18           Rivadavia   $148,125     $2,849
19         River Plate   $987,656    $18,993
20     Rosario Central   $319,655     $6,147
21        

In [19]:
from google.cloud import bigquery

este seria el codigo equivalente para subir a bigquery pero no se si lo necesitamos aun

In [22]:
# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.primera_division'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

ValueError: This method requires pyarrow to be installed